In [1]:
from lbmpy.session import *
from lbmpy.phasefield.n_phase_boyer import *
from lbmpy.phasefield.analytical import *
from itertools import permutations

# A) Homogenous surface tensions case (3.1)

Equation numbers refer to paper *"Hierarchy of consistent n-component Cahn-Hilliard systems"* by Franck Boyer, Sebastian Minjeaud


## 1) Testing properties of $\bar{\alpha}$

testing if equation (3.1) is indeed fulfilled

In [2]:
for n in [2, 3, 4, 5, 9]:  # test for various number of phases
    σ_sym = sp.symbols("sigma")
    σ = sp.ImmutableDenseMatrix(n, n, lambda i, j: σ_sym if i != j else 0)
    α_bar, _ = diffusion_coefficients(σ)
    for i in range(n):
        for j in range(n):
            if i != j:
                assert α_bar[i, j] == 1 / (n * σ_sym)
            else:
                assert α_bar[i, j] == -(n-1) / (n * σ_sym)

## 2) Testing properties of $\Psi_k^{[n]}$

Proposition (3.1) in the paper

In [3]:
for n in [2, 3, 7]:
    c = sp.symbols("c_:{n}".format(n=n-1))
    f = lambda c: c**2 * (1-c)**2
    for k in range(1, n):
        assert sp.expand( psi(k, c + (0,), f) - psi(k, c, f) ) == 0  # Proposition 3.1 (i)
    assert psi(n, c + (0,), f) == 0

### 3) Assemble free energy and check necessary properties

In [4]:
n = 5
c = sp.symbols("c_:{n}".format(n=n))
f = lambda c: c**2 * (1-c)**2

sigma = sp.symbols("sigma")

f_bulk = capital_f_bulk_equal_surface_tension(c, f, sigma, 1)

σ_mag = sp.ImmutableDenseMatrix(n, n, lambda i, j: sigma if i != j else 0)
lb = l_bar(f_bulk, diffusion_coefficients(σ_mag)[0], c)

for i, lb_i in enumerate(lb):
    if i != n-1:
        term = lb_i.subs(c[-1], 1 - sum(c[:-1])).subs(c[i], 0) 
    else:
        term = lb_i.subs(c[-1], 0).subs(c[-2], 1 - sum(c[:-2]))
    assert sp.expand(term) == 0

## Testing Proposition 3.3

first line of Proposition 3.3

In [5]:
for n in [2, 3, 4, 5, 6, 10]:
    f = lambda c: c**2 * (1-c)**2
    c = sp.symbols("c_:{n}".format(n=n-1))
    c = c + (1 - sum(c),)

    result = psi(1, c, f) + psi(2, c, f)
    if n in (2, 3):
        assert result == 0
    else:
        expected = 24 * sum(c[i[0]] * c[i[1]] * c[i[2]] * c[i[3]] 
                            for i in capital_i(4, n))
        assert sp.expand(result - expected) == 0

last line

In [6]:
def compare_f_bulk(n):
    sigma = sp.symbols("sigma")
    f = lambda c: c**2 * (1-c)**2
    c = sp.symbols("c_:{n}".format(n=n-1))
    c = c + (1 - sum(c),)
    own = capital_f_bulk_equal_surface_tension(c, f, sigma, 1)
    if n == 2:
        ref = sigma * f(c[0])
    if n == 3:
        ref = sigma / 2 * ( f(c[0]) + f(c[1]) + f(c[2]) )
    else:
        ref = ( sigma / 2 * sum(f(c_k) for c_k in c) + 
                2 * sigma * sum(c[i[0]] * c[i[1]] * c[i[2]] * c[i[3]] for i in capital_i(4, n) ) )
    
    assert sp.expand(ref - own) == 0

for n in range(2, 7):
    print("Testing n =", n, end=':')
    compare_f_bulk(n)
    print("  ok")

Testing n = 2:  ok
Testing n = 3:  ok
Testing n = 4:  ok
Testing n = 5:  ok
Testing n = 6:  ok


# B) Arbitrary surface tension case (3.2)

In [7]:
def numeric_surface_tensions(n):
    """Some numeric values for surface tensions - symbolic values take too long"""
    return sp.ImmutableDenseMatrix(n, n, lambda i, j: 0 if i == j else (i+1) * (j+1))

Checking consistency with 2-phase system

Makes sure (C2) and (C3) are satisfied if $|I|=n-2$

In [8]:
for n in [3, 4, 5, 8]:
    print("n =", n)
    c = sp.symbols(f"c_:{n}")
    σ = numeric_surface_tensions(n)
    α, γ = diffusion_coefficients(σ)
    f0 = capital_f0(c, σ, lambda c: c**2 * (1-c)**2)
    
    pairs_to_test = combinations(range(n), 2) if n < 5 else [(0, n-1), (1, 3), (1, 4)]
    for i0, j0 in pairs_to_test:
        print("  Testing", i0, j0, end=' : ')

        substitutions = {c[i]: 0 for i in range(n) if i not in (i0, j0)}
        substitutions[c[j0]] = 1 - c[i0]
        l = sp.expand(l_bar(f0, α, c).subs(substitutions))
        for i in range(n):
            if i not in (i0, j0):
                assert l[i] == 0
        print("OK")

n = 3
  Testing 0 1 : OK
  Testing 0 2 : OK
  Testing 1 2 : OK
n = 4
  Testing 0 1 : OK
  Testing 0 2 : OK
  Testing 0 3 : OK
  Testing 1 2 : OK
  Testing 1 3 : OK
  Testing 2 3 : OK
n = 5
  Testing 0 4 : OK
  Testing 1 3 : OK
  Testing 1 4 : OK
n = 8
  Testing 0 7 : OK
  Testing 1 3 : OK
  Testing 1 4 : OK


Checking consistency with 3-phase system

In [9]:
for n in [4, 6]:
    print("n=", n)
    c = sp.symbols(f"c_:{n}", real=True)
    σ = numeric_surface_tensions(n)
    α, γ = diffusion_coefficients(σ)
    f0 = capital_f0(c, σ, lambda c: c**2 * (1-c)**2) + correction_g(c, σ)
    
    triples_to_test = combinations(range(n), 3) if n < 5 else [(0, n-2, n-1), (1,2, 3), (0, 1, 4)]
    for ind in triples_to_test:
        print("  Testing", ind, end=' : ')

        substitutions = {c[i]: 0 for i in range(n) if i not in ind}
        substitutions[c[ind[2]]] = 1 - c[ind[0]] - c[ind[1]]

        l = l_bar(f0, α, c).subs(substitutions)
        for i in range(n):
            if i not in ind:
                back_substitutions = { 1 - c[ind[0]] - c[ind[1]]: c[ind[2]] }
                for c_i in c:
                    back_substitutions[c_i] = sp.Symbol(c_i.name, positive=True)
                l_i = sp.simplify(l[i]).subs(back_substitutions)
                assert l_i == 0
        print("OK")

n= 4
  Testing (0, 1, 2) : OK
  Testing (0, 1, 3) : OK
  Testing (0, 2, 3) : OK
  Testing (1, 2, 3) : OK
n= 6
  Testing (0, 4, 5) : OK
  Testing (1, 2, 3) : OK
  Testing (0, 1, 4) : OK


Check explicit formula for $\Theta$ in case of $n=4$

In [10]:
def theta4(alpha, ind):
    assert len(ind) == 4
    assert alpha.rows == 4
    k, l = ind[2], ind[3]
    return 2 * alpha[k, l] / alpha[k, k]

n = 4
c = sp.symbols(f"c_:{n}")
σ = numeric_surface_tensions(n)
α, γ = diffusion_coefficients(σ)

for ind in permutations(range(4)):
    assert capital_theta(α, ind) == theta4(α, ind), "Check failed for " + str(ind)

# C) Interface width and surface tension check on binary interface

In [11]:
from lbmpy.phasefield.n_phase_boyer import free_energy as free_energy_boyer
import pystencils as ps
num_phases = 3

epsilon = 5
x = sp.symbols("x")
# Interface width is defined differently in this paper: i.e. (usual eps) = (their eps) / 4
c_a = analytic_interface_profile(x, interface_width= epsilon * sp.Rational(1,4))
c = sp.symbols("c_:{}".format(num_phases))

In [12]:
sigma = symbolic_surface_tensions(num_phases)
F = free_energy_boyer(c, epsilon=epsilon, surface_tensions=sigma, stabilization_factor=0)
mu = chemical_potentials_from_free_energy(F, c)

In [13]:
# Check all permutations of phases
for i in range(num_phases):
    for j in range(num_phases):
        if i == j:
            continue
        print("  -> Testing interface between", i, "and", j)
        substitutions = {c_i: 0 for c_i in c}
        substitutions[c[i]] = c_a
        substitutions[c[j]] = 1 - c_a

        for μ_i in mu:
            res = ps.fd.evaluate_diffs(μ_i.subs(substitutions), x).expand()
            assert res == 0, "Analytic interface profile wrong for phase between %d and %d" % (i, j)

        two_phase_free_energy = F.subs(substitutions)
        two_phase_free_energy = sp.simplify(ps.fd.evaluate_diffs(two_phase_free_energy, x))
        result = cosh_integral(two_phase_free_energy, x)
        assert result == sigma[i, j]
print("Done")        

  -> Testing interface between 0 and 1
  -> Testing interface between 0 and 2
  -> Testing interface between 1 and 0
  -> Testing interface between 1 and 2
  -> Testing interface between 2 and 0
  -> Testing interface between 2 and 1
Done
